

# <h3 align="center"> $\underline{\text{ Large Language Model (LLM) }}$</h3>

<h3 align="center">$ \text{Artificial Intelligence (AI) algorithm that uses deep learning techniques.}$</h3>








Training a Transformer based language model:
Character level language model

Dataset: Pablo Neruda -> 20 poemas de amor y una canción desesperada.

In [11]:
with open('neruda_20poemasyCD.txt','r',encoding='utf-8') as f:
  text = f.read()

In [12]:
print("lenth of dataset in characteres: ", len(text))

lenth of dataset in characteres:  21207


In [13]:
print(text[:100])

Poema 1

Cuerpo de mujer, blancas colinas, muslos blancos,
te pareces al mundo en tu actitud de entr


In [14]:
#unique characters enumeration:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"(),-.0123456789:;?ABCDEFGHIJLMNOPQRSTUVYZabcdefghijlmnopqrstuvxyzÁÚáéíñóú
78


# **Tokenizers**

Character level Tokinizer

In [15]:
# simple one: mapping from Char to Int and back
chToIn = {c:i for i,c in enumerate(chars)}
inToCh = {i:c for i,c in enumerate (chars)}
encod = lambda x: [chToIn[c] for c in x]          #char_list -> int
decod = lambda y: ''.join([inToCh[i]for i in y])  #int -> char_list
print(encod("Cuerpo de mujer"))
print(decod(encod("blancas colinas")))

[25, 65, 50, 62, 60, 59, 2, 49, 50, 2, 57, 65, 55, 50, 62]
blancas colinas


Subword level Tokinizer

In [16]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer.tokenize(text[:40])

['poem',
 '##a',
 '1',
 'cue',
 '##rp',
 '##o',
 'de',
 'mu',
 '##jer',
 ',',
 'blanca',
 '##s',
 'colin',
 '##a']

Word level Tokinizer

In [18]:
!pip install -q "tensorflow-text==2.11.*"

In [19]:
import tensorflow_text as tf_text
tokenizer = tf_text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(text)
print(tokens[:40])

tf.Tensor(
[b'Poema' b'1' b'Cuerpo' b'de' b'mujer,' b'blancas' b'colinas,' b'muslos'
 b'blancos,' b'te' b'pareces' b'al' b'mundo' b'en' b'tu' b'actitud' b'de'
 b'entrega.' b'Mi' b'cuerpo' b'de' b'labriego' b'salvaje' b'te' b'socava'
 b'y' b'hace' b'saltar' b'el' b'hijo' b'del' b'fondo' b'de' b'la'
 b'tierra.' b'Fui' b'solo' b'como' b'un' b't\xc3\xbanel.'], shape=(40,), dtype=string)


Tokenizing the entire trainig set in PyTorch

In [20]:
import torch
data = torch.tensor(encod(text),dtype=torch.long) #store ina torch.tensor
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([21207]) torch.int64
tensor([37, 59, 50, 57, 46,  2, 11,  0,  0, 25, 65, 50, 62, 60, 59,  2, 49, 50,
         2, 57, 65, 55, 50, 62,  7,  2, 47, 56, 46, 58, 48, 46, 63,  2, 48, 59,
        56, 54, 58, 46, 63,  7,  2, 57, 65, 63, 56, 59, 63,  2, 47, 56, 46, 58,
        48, 59, 63,  7,  0, 64, 50,  2, 60, 46, 62, 50, 48, 50, 63,  2, 46, 56,
         2, 57, 65, 58, 49, 59,  2, 50, 58,  2, 64, 65,  2, 46, 48, 64, 54, 64,
        65, 49,  2, 49, 50,  2, 50, 58, 64, 62, 50, 52, 46,  9,  0, 34, 54,  2,
        48, 65, 50, 62, 60, 59,  2, 49, 50,  2, 56, 46, 47, 62, 54, 50, 52, 59,
         2, 63, 46, 56, 66, 46, 55, 50,  2, 64, 50,  2, 63, 59, 48, 46, 66, 46,
         0, 68,  2, 53, 46, 48, 50,  2, 63, 46, 56, 64, 46, 62,  2, 50, 56,  2,
        53, 54, 55, 59,  2, 49, 50, 56,  2, 51, 59, 58, 49, 59,  2, 49, 50,  2,
        56, 46,  2, 64, 54, 50, 62, 62, 46,  9,  0,  0, 28, 65, 54,  2, 63, 59,
        56, 59,  2, 48, 59, 57, 59,  2, 65, 58,  2, 64, 77, 58, 50, 56,  9,  2,
        

Split data into train and validation

In [21]:
n = int(0.9*len(data)) #90% = Train, 10 = Val
trainD = data[:n]
valD = data[n:]

#Data Loader

In [22]:
blockSize = 8
trainD[:blockSize+1]

tensor([37, 59, 50, 57, 46,  2, 11,  0,  0])

Time dimension of the tensors that are going to be feeding into the transformer:

In [23]:
x = trainD[:blockSize]
y = trainD[1:blockSize+1]
for t in range(blockSize):
  contxt = x[:t+1]
  targt = y[t]
  print(f"Input: {contxt} -> Target: {targt}")

Input: tensor([37]) -> Target: 59
Input: tensor([37, 59]) -> Target: 50
Input: tensor([37, 59, 50]) -> Target: 57
Input: tensor([37, 59, 50, 57]) -> Target: 46
Input: tensor([37, 59, 50, 57, 46]) -> Target: 2
Input: tensor([37, 59, 50, 57, 46,  2]) -> Target: 11
Input: tensor([37, 59, 50, 57, 46,  2, 11]) -> Target: 0
Input: tensor([37, 59, 50, 57, 46,  2, 11,  0]) -> Target: 0


Batch dimension: multiple chuncks of text stacked up in a single tensor (4x8)

In [24]:
torch.manual_seed(1337)
batch = 4
block = 8
def getBatch(split):
  data = trainD if split == 'train' else valD
  ix = torch.randint(len(data) - block, (batch,))
  x = torch.stack([data[i:i+block]for i in ix])
  y = torch.stack([data[i+1:i+block+1]for i in ix])
  return x,y

xb, yb = getBatch('train')
print('input ')
print(xb.shape)
print(xb,"\n")
print('target ')
print(yb.shape)
print(yb,"\n")
for b in range(batch):
  for t in range(block):
    contxt = xb[b,:t+1]
    targt = yb [b,t]
    print(f"Input: {contxt.tolist()} Target -> {targt}")

input 
torch.Size([4, 8])
tensor([[62, 50, 58, 64, 50,  2, 46, 56],
        [58,  2, 61, 65, 50,  2, 57, 54],
        [65, 55, 50, 62,  7,  2, 47, 56],
        [63,  9,  0, 43, 46, 58,  2, 64]]) 

target 
torch.Size([4, 8])
tensor([[50, 58, 64, 50,  2, 46, 56,  2],
        [ 2, 61, 65, 50,  2, 57, 54,  2],
        [55, 50, 62,  7,  2, 47, 56, 46],
        [ 9,  0, 43, 46, 58,  2, 64, 62]]) 

Input: [62] Target -> 50
Input: [62, 50] Target -> 58
Input: [62, 50, 58] Target -> 64
Input: [62, 50, 58, 64] Target -> 50
Input: [62, 50, 58, 64, 50] Target -> 2
Input: [62, 50, 58, 64, 50, 2] Target -> 46
Input: [62, 50, 58, 64, 50, 2, 46] Target -> 56
Input: [62, 50, 58, 64, 50, 2, 46, 56] Target -> 2
Input: [58] Target -> 2
Input: [58, 2] Target -> 61
Input: [58, 2, 61] Target -> 65
Input: [58, 2, 61, 65] Target -> 50
Input: [58, 2, 61, 65, 50] Target -> 2
Input: [58, 2, 61, 65, 50, 2] Target -> 57
Input: [58, 2, 61, 65, 50, 2, 57] Target -> 54
Input: [58, 2, 61, 65, 50, 2, 57, 54] Target -> 2

Input to the Transformer:

In [25]:
print(xb)

tensor([[62, 50, 58, 64, 50,  2, 46, 56],
        [58,  2, 61, 65, 50,  2, 57, 54],
        [65, 55, 50, 62,  7,  2, 47, 56],
        [63,  9,  0, 43, 46, 58,  2, 64]])


#Neural Network: Bigram Language Model

1) Simplest Language Model
- The tockens are not talking to each other

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decod(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 78])
tensor(4.6538, grad_fn=<NllLossBackward0>)

I(;xxÚfvyc?QP(SA ílHDCUgBep6S2ñ6N)Gc FPyCcl,Á7hdFn3q!
SP3;xñ0ÁerSOt2
ái"(HzESZCx:rNRóH0bEeIi3:C
5ÚJ


#Training the Model

Optimizer

In [27]:
opt = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [28]:
batch = 32
for stp in range(10000):
  xb,yb = getBatch('train')
  log, loss= m(xb,yb)
  opt.zero_grad(set_to_none=True)
  loss.backward()
  opt.step()

print(loss.item())

2.2536509037017822


Result:

In [29]:
print(decod(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Perestivitunco.

Bdos lgr, motamQueres vebare uí.
Y pe s, a tuzBU3aú deso seniscocla ndemi os y entú la ltrintues.
yasa, bro s

Eni lmorbllasechSe enimbl momíAs RFGis a, mosJVigegrtúsu))uejen So. 4
y spas, te.
Lan erel vasquedola lltoY e, lmua n delge lasquara n to ma qus
P8ás.
Cula emeabrre s es e)í.


Y lambocagra tos e heren ie tea.
Asies brere spasinmo strpergun quiegaQEro57
toste carons fisostentoschoón ceruete s.

UUO tara R3quempe o ve n y pes.
y nischoía poso cumeraures
homin tr mertes e
